In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, LeakyReLU, MaxPooling2D, Cropping2D
from tensorflow.keras.layers import Conv2DTranspose, BatchNormalization, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np

In [14]:
# Configura el ImageDataGenerator para el entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normaliza las imágenes
    rotation_range=20,  # Añade algo de rotación para robustez
    width_shift_range=0.2,  # Desplazamientos horizontales
    height_shift_range=0.2,  # Desplazamientos verticales
    shear_range=0.2,  # Cizallamiento
    zoom_range=0.2,  # Zoom
    horizontal_flip=True,  # Invierte las imágenes horizontalmente
    fill_mode='nearest'
)

# Suponiendo que tus carpetas se llaman 'not_manipulated' y 'manipulated'
train_generator = train_datagen.flow_from_directory(
    'D:\dataset--ml\CASIA2-Train',  # Asegúrate de que esta ruta es accesible y correcta
    target_size=(150, 150),  # Redimensiona las imágenes a 150x150
    batch_size=32,
    class_mode='binary'  # 'binary' porque tienes dos clases, manipulado y no manipulado
)


Found 9614 images belonging to 2 classes.


In [57]:
def build_discriminator(input_shape):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')
    
    base_model.trainable = True

    fine_tune_at = 100

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal'),
        tf.keras.layers.RandomRotation(0.2),
    ])

    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    #x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)

    # model = Sequential([
    #     Conv2D(64, (5, 5), padding='same', input_shape=input_shape),
    #     LeakyReLU(alpha=0.2),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     Conv2D(128, (5, 5)),
    #     LeakyReLU(alpha=0.2),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     Flatten(),
    #     Dense(50, activation='sigmoid'),
    #     Dense(1, activation='sigmoid')
    # ])

    # def custom_loss(y_true, y_pred):
    #     # Penaliza las salidas cercanas a 0.5
    #     penalty = tf.abs(y_pred - 0.5)
    #     return tf.keras.losses.binary_crossentropy(y_true, y_pred) + penalty

    #model.compile(loss=custom_loss, optimizer='adam', metrics=['accuracy'])
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00001),
              metrics=['accuracy'])
    
    return model

discriminator = build_discriminator((150, 150, 3))

In [ ]:
# Entrenar el discriminador
history = discriminator.fit(
    train_generator,
    epochs=10
)

In [ ]:
# Configura el ImageDataGenerator para el entrenamiento
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = train_datagen.flow_from_directory(
    'D:\dataset--ml\CASIA2-Test',  # Asegúrate de que esta ruta es accesible y correcta
    target_size=(150, 150),  # Redimensiona las imágenes a 150x150
    batch_size=32,
    class_mode='binary'  # 'binary' porque tienes dos clases, manipulado y no manipulado
)

loss, accuracy = discriminator.evaluate(test_generator, steps=len(test_generator))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [49]:
def build_generator(latent_dim):
    model = Sequential([
        # Asegúrate de que la salida de la capa densa se redimensiona correctamente
        Dense(128 * 38 * 38, activation="relu", input_dim=latent_dim),  # Ajuste basado en el cálculo a continuación
        Reshape((38, 38, 128)),  # Comienza desde una base que pueda escalarse correctamente
        BatchNormalization(),

        # Primera capa transpuesta para escalar de 38x38 a 76x76
        Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu'),
        BatchNormalization(),

        # Segunda capa transpuesta, escala de 76x76 a 152x152, necesitaremos recortar después
        Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu'),
        BatchNormalization(),

        # Capa final para ajustar a 150x150 y reducir a 3 canales para la imagen RGB
        Conv2DTranspose(3, kernel_size=3, strides=1, padding='valid', activation='sigmoid'),  # Usando 'valid' para ajustar exactamente
        # Recorte las dimensiones para que coincidan con 150x150x3
        Cropping2D(cropping=((2, 2), (2, 2)))  # Recorta 1 pixel de cada borde
    ])
    return model

latent_dim = 100
generator = build_generator(latent_dim)

In [50]:
def build_gan(generator, discriminator):
    # Asegurarse de que el discriminador no sea entrenable cuando se entrena la GAN
    discriminator.trainable = False

    # Entrada de ruido al generador
    z = Input(shape=(latent_dim,))
    # Imagen generada
    img = generator(z)
    # Discriminador decide si la imagen es real o falsa
    valid = discriminator(img)

    # Modelo completo
    model = Model(inputs=z, outputs=valid)
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    
    return model

# Crea la GAN combinando generador y discriminador
gan = build_gan(generator, discriminator)

In [9]:
import matplotlib.pyplot as plt

In [53]:
def train_gan(generator, discriminator, gan, epochs, batch_size, save_interval):
    for epoch in range(epochs):
        # Obtener un lote de imágenes no manipuladas
        real_images, labels = next(train_gan_generator)
        length = len(real_images)

        # Generar ruido aleatorio
        noise = np.random.normal(0, 1, (length, latent_dim))

        # Generar un lote de nuevas imágenes
        generated_images = generator.predict(noise)

        # Preparar etiquetas para el lote real y falso
        valid = np.ones((length, 1))
        fake = np.zeros((length, 1))

        # Entrenar el discriminador (real clasificado como 1 y falso como 0)
        d_loss_real = discriminator.train_on_batch(real_images, labels)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Entrenar el generador (tratamos de engañar al discriminador para clasificar falsos como reales)
        g_loss = gan.train_on_batch(noise, valid)

        # Progreso de impresión y guardado ocasional de imágenes
        print(f"Epoch {epoch + 1}/{epochs} - D loss: {d_loss} - G loss: {g_loss}")
        
        if (epoch + 1) % save_interval == 0:
            #save_images(epoch, generator)
            pass

def save_images(epoch, generator, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, (dim[0] * dim[1], latent_dim))
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(dim[0] * dim[1], 150, 150, 3)
    
    # Configurar la cuadrícula de imágenes
    plt.figure(figsize=figsize)
    for i in range(dim[0] * dim[1]):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')  # No mostrar ejes
    
    plt.tight_layout()  # Ajustar el layout para minimizar el espacio entre imágenes
    plt.show()  # Mostrar todas las imágenes generadas


# Configurar parámetros
epochs = 65
batch_size = 32
save_interval = 10

train_gan_datagen = ImageDataGenerator(rescale=1./255)
train_gan_generator = train_datagen.flow_from_directory(
    'D:\dataset--ml\CASIA2-Train',  # Asegúrate de que apunta a la carpeta correcta
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary',  # Sin etiquetas ya que sólo cargamos imágenes
    shuffle=True
)

# Llamar a train_gan
train_gan(generator, discriminator, gan, epochs, batch_size, save_interval)

Found 9614 images belonging to 2 classes.
1/1 [==============================] - 1s 566ms/step
Epoch 1/65 - D loss: [2.26146752 0.34375   ] - G loss: 15.424948692321777
1/1 [==============================] - 1s 631ms/step
Epoch 2/65 - D loss: [0.90408236 0.3125    ] - G loss: 15.424948692321777
1/1 [==============================] - 1s 640ms/step
Epoch 3/65 - D loss: [0.55435243 0.828125  ] - G loss: 15.424948692321777
1/1 [==============================] - 1s 645ms/step
Epoch 4/65 - D loss: [0.44555144 0.859375  ] - G loss: 15.424948692321777
1/1 [==============================] - 1s 617ms/step
Epoch 5/65 - D loss: [0.45562802 0.8125    ] - G loss: 15.424948692321777
1/1 [==============================] - 1s 647ms/step
Epoch 6/65 - D loss: [0.40338811 0.859375  ] - G loss: 15.424948692321777
1/1 [==============================] - 1s 621ms/step
Epoch 7/65 - D loss: [0.38183238 0.828125  ] - G loss: 15.424948692321777
1/1 [==============================] - 1s 717ms/step
Epoch 8/65 - D l

In [54]:
# Configura el ImageDataGenerator para el entrenamiento
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = train_datagen.flow_from_directory(
    'D:\dataset--ml\CASIA2-Test',  # Asegúrate de que esta ruta es accesible y correcta
    target_size=(150, 150),  # Redimensiona las imágenes a 150x150
    batch_size=32,
    class_mode='binary'  # 'binary' porque tienes dos clases, manipulado y no manipulado
)

loss, accuracy = discriminator.evaluate(test_generator, steps=len(test_generator))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Found 3000 images belonging to 2 classes.
94/94 [==============================] - 48s 508ms/step - loss: 0.7537 - accuracy: 0.5000
Test Loss: 0.7536779046058655
Test Accuracy: 0.5


In [56]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Cargar y preparar la imagen
def prepare_image(image_path, target_size=(150, 150)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalizar a 0-1
    img_array = np.expand_dims(img_array, axis=0)  # Añadir batch dimension
    return img_array

image_path = 'D:\dataset--ml\CASIA2-Train\Au\Au_ani_00036.jpg'
prepared_image = prepare_image(image_path)

def predict_image_realness(model, image):
    prediction = model.predict(image)
    return prediction[0][0]  # Devuelve la probabilidad de que la imagen sea real

# Suponiendo que `discriminator` es tu modelo entrenado
realness_prob = predict_image_realness(discriminator, prepared_image)
print(f"Probabilidad de que la imagen sea falsa: {realness_prob:.2f}")


1/1 [==============================] - 1s 1s/step
Probabilidad de que la imagen sea falsa: -0.64
